<h1> Repeatable splitting </h1>

In this notebook, we will explore the impact of different ways of creating machine learning datasets.

<p>

Repeatability is important in machine learning. If you do the same thing now and 5 minutes from now and get different answers, then it makes experimentation difficult. In other words, you will find it difficult to gauge whether a change you made has resulted in an improvement or not.

In [1]:
import google.datalab.bigquery as bq

<h3> Create a simple machine learning model </h3>

The dataset that we will use is <a href="https://bigquery.cloud.google.com/table/bigquery-samples:airline_ontime_data.flights">a BigQuery public dataset</a> of airline arrival data. Click on the link, and look at the column names. Switch to the Details tab to verify that the number of records is 70 million, and then switch to the Preview tab to look at a few rows.
<p>
We want to predict the arrival delay of an airline based on the departure delay. The model that we will use is a zero-bias linear model:
$$ delay_{arrival} = \alpha * delay_{departure} $$
<p>
To train the model is to estimate a good value for $\alpha$. 
<p>
One approach to estimate alpha is to use this formula:
$$ \alpha = \frac{\sum delay_{departure} delay_{arrival} }{  \sum delay_{departure}^2 } $$
Because we'd like to capture the idea that this relationship is different for flights from New York to Los Angeles vs. flights from Austin to Indianapolis (shorter flight, less busy airports), we'd compute a different $alpha$ for each airport-pair.  For simplicity, we'll do this model only for flights between Denver and Los Angeles.

<h2> Naive random split (not repeatable) </h2>

In [4]:
compute_alpha = """
#standardSQL

   SELECT RAND() AS splitfield,
   arrival_delay,
   departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'

"""

In [5]:
results = bq.Query(compute_alpha).execute().result().to_dataframe()
#prod = results['prod'][0]
#print(prod)
print(results)  # basically you are giving it an index using a random probability distribution

       splitfield  arrival_delay  departure_delay
0        0.545383           35.0             10.0
1        0.487522           30.0             19.0
2        0.938933           29.0             18.0
3        0.891664           29.0             29.0
4        0.547597           52.0             57.0
5        0.950596           30.0             33.0
6        0.709231           38.0             22.0
7        0.110976           31.0             28.0
8        0.211616           85.0             72.0
9        0.007159           42.0             30.0
10       0.394859           27.0             45.0
11       0.117991           48.0             39.0
12       0.652870           62.0             52.0
13       0.750834           58.0             74.0
14       0.717552          176.0            179.0
15       0.150757           42.0             -3.0
16       0.484473           68.0             80.0
17       0.521613           29.0             14.0
18       0.417239          113.0            111.0


In [6]:
compute_alpha = """
#standardSQL
SELECT 
   arrival_delay * departure_delay AS prod
FROM
(
   SELECT RAND() AS splitfield,
   arrival_delay,
   departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
)
WHERE
  splitfield < 0.8
"""

In [7]:
results = bq.Query(compute_alpha).execute().result().to_dataframe()
prod = results['prod'][0]
print(prod)
print(results)

216.0
         prod
0       216.0
1       -70.0
2       350.0
3       -50.0
4       460.0
5       -33.0
6       -40.0
7       570.0
8       775.0
9       522.0
10      841.0
11     2964.0
12      -72.0
13      -55.0
14      990.0
15      836.0
16       63.0
17       49.0
18      868.0
19      208.0
20      -28.0
21      -21.0
22     1260.0
23      112.0
24      -36.0
25      152.0
26      322.0
27     1215.0
28      176.0
29       99.0
...       ...
63595    90.0
63596    90.0
63597    90.0
63598    90.0
63599    90.0
63600    90.0
63601    90.0
63602    90.0
63603    90.0
63604    90.0
63605    90.0
63606    90.0
63607    90.0
63608    90.0
63609    90.0
63610    90.0
63611    90.0
63612    90.0
63613    90.0
63614    90.0
63615    90.0
63616    90.0
63617    90.0
63618    90.0
63619    90.0
63620    90.0
63621    90.0
63622    90.0
63623    90.0
63624    90.0

[63625 rows x 1 columns]


In [8]:

compute_alpha = """
#standardSQL
SELECT 
   SUM(arrival_delay * departure_delay) AS sum
FROM
(
   SELECT RAND() AS splitfield,
   arrival_delay,
   departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
)
WHERE
  splitfield < 0.8
"""

In [9]:
results = bq.Query(compute_alpha).execute().result().to_dataframe()
sum = results['sum'][0]
print(sum)
print(results)

62045782.0
          sum
0  62045782.0


In [10]:
compute_alpha = """
#standardSQL
SELECT 
   SAFE_DIVIDE(SUM(arrival_delay * departure_delay), SUM(departure_delay * departure_delay)) AS alpha
FROM
(
   SELECT RAND() AS splitfield,
   arrival_delay,
   departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
)
WHERE
  splitfield < 0.8
"""

In [11]:
results = bq.Query(compute_alpha).execute().result().to_dataframe()
alpha = results['alpha'][0]
print(alpha)

0.976171791442077


<h3> What is wrong with calculating RMSE on the training and test data as follows? </h3>

In [14]:
compute_rmse = """
#standardSQL

  SELECT
    IF (RAND() < 0.8, 'train', 'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' 
"""
bq.Query(compute_rmse).execute().result()

dataset,arrival_delay,departure_delay
train,35.0,10.0
train,30.0,19.0
eval,29.0,18.0
train,29.0,29.0
eval,52.0,57.0
eval,30.0,33.0
train,38.0,22.0
train,31.0,28.0
train,85.0,72.0
eval,42.0,30.0


In [15]:
compute_rmse = """
#standardSQL
SELECT
  dataset,
  SQRT(AVG((arrival_delay - ALPHA * departure_delay)*(arrival_delay - ALPHA * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM (
  SELECT
    IF (RAND() < 0.8, 'train', 'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' )
GROUP BY
  dataset
"""
bq.Query(compute_rmse.replace('ALPHA', str(alpha))).execute().result()

dataset,rmse,num_flights
train,13.104918736934946,63491
eval,13.000627661275605,16198


Hint:
* Are you really getting the same training data in the compute_rmse query as in the compute_alpha query?
* Do you get the same answers each time you rerun the compute_alpha and compute_rmse blocks?

<h3> How do we correctly train and evaluate? </h3>
<br/>
Here's the right way to compute the RMSE using the actual training and held-out (evaluation) data. Note how much harder this feels.

Although the calculations are now correct, the experiment is still not repeatable.

Try running it several times; do you get the same answer?

In [16]:

train_and_eval_rand = """
#standardSQL
WITH
  alldata AS (
  SELECT
    IF (RAND() < 0.8,
      'train',
      'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' )
SELECT
  dataset,
  COUNT(arrival_delay) AS num_flights
FROM
  alldata
GROUP BY
  dataset
"""

In [17]:
bq.Query(train_and_eval_rand).execute().result()

dataset,num_flights
train,63817
eval,15872


In [18]:
train_and_eval_rand = """
#standardSQL
WITH
  alldata AS (
  SELECT
    IF (RAND() < 0.8,
      'train',
      'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' ),
  training AS (
  SELECT
    SAFE_DIVIDE( SUM(arrival_delay * departure_delay) , SUM(departure_delay * departure_delay)) AS alpha
  FROM
    alldata
  WHERE
    dataset = 'train' )
SELECT
  MAX(alpha) AS alpha,
  dataset,
  SQRT(AVG((arrival_delay - alpha * departure_delay)*(arrival_delay - alpha * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM
  alldata,
  training
GROUP BY
  dataset
"""

In [19]:
bq.Query(train_and_eval_rand).execute().result()

alpha,dataset,rmse,num_flights
0.9760207869303448,train,13.084922918858021,63895
0.9760207869303448,eval,13.079169661146635,15794


<h2> Using HASH of date to split the data </h2>

Let's split by date and train.

In [24]:
compute_alpha = """
#standardSQL
SELECT 
    *
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
  AND MOD(ABS(FARM_FINGERPRINT(date)), 10) < 8
LIMIT 10
"""
results = bq.Query(compute_alpha).execute().result().to_dataframe()
print(results)

         date airline airline_code departure_airport departure_state  \
0  2012-11-12      F9        20436               DEN              CO   
1  2012-11-01      F9        20436               DEN              CO   
2  2012-11-12      F9        20436               DEN              CO   
3  2012-11-12      F9        20436               DEN              CO   
4  2012-11-21      F9        20436               DEN              CO   
5  2012-01-10      F9        20436               DEN              CO   
6  2012-11-12      F9        20436               DEN              CO   
7  2012-01-08      F9        20436               DEN              CO   
8  2012-11-01      F9        20436               DEN              CO   
9  2012-11-12      F9        20436               DEN              CO   

   departure_lat  departure_lon arrival_airport arrival_state  arrival_lat  \
0          39.86        -104.67             LAX            CA        33.94   
1          39.86        -104.67             LAX    

In [25]:
compute_alpha = """
#standardSQL
SELECT 
    FARM_FINGERPRINT(date)
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
  AND MOD(ABS(FARM_FINGERPRINT(date)), 10) < 8
LIMIT 10
"""
results = bq.Query(compute_alpha).execute().result().to_dataframe()
print(results)

                   f0_
0  3725999277042568035
1  4810755874896543010
2  3725999277042568035
3  3725999277042568035
4  8090764422093330687
5  4456799817030954973
6  3725999277042568035
7  4561376087784110635
8  4810755874896543010
9  3725999277042568035


In [26]:
compute_alpha = """
#standardSQL
SELECT 
    FARM_FINGERPRINT(date)
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
  AND MOD(ABS(FARM_FINGERPRINT(date)), 10) > 8
LIMIT 10
"""
results = bq.Query(compute_alpha).execute().result().to_dataframe()
print(results)

                   f0_
0 -1556909689059132759
1 -1556909689059132759
2 -3426016097554679449
3 -3426016097554679449
4 -3426016097554679449
5 -3426016097554679449
6 -3145252244108862159
7 -3145252244108862159
8 -3145252244108862159
9 -3145252244108862159


In [27]:
compute_alpha = """
#standardSQL
SELECT 
   SAFE_DIVIDE(SUM(arrival_delay * departure_delay), SUM(departure_delay * departure_delay)) AS alpha
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
  AND MOD(ABS(FARM_FINGERPRINT(date)), 10) < 8
"""
results = bq.Query(compute_alpha).execute().result().to_dataframe()
alpha = results['alpha'][0]
print(alpha)

0.9758039143620403


We can now use the alpha to compute RMSE. Because the alpha value is repeatable, we don't need to worry that the alpha in the compute_rmse will be different from the alpha computed in the compute_alpha.

In [32]:
compute_rmse = """
#standardSQL
SELECT
  IF(MOD(ABS(FARM_FINGERPRINT(date)), 10) < 8, 'train', 'eval') AS dataset,
  SQRT(AVG((arrival_delay - ALPHA * departure_delay)*(arrival_delay - ALPHA * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM
    `bigquery-samples.airline_ontime_data.flights`
WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX'
GROUP BY
  dataset
"""
print(bq.Query(compute_rmse.replace('ALPHA', str(alpha))).execute().result().to_dataframe().head())

  dataset       rmse  num_flights
0    eval  12.764685        15671
1   train  13.160712        64018


Note also that the RMSE on the evaluation dataset more from the RMSE on the training dataset when we do the split correctly.  This should be expected; in the RAND() case, there was leakage between training and evaluation datasets, because there is high correlation between flights on the same day.
<p>
This is one of the biggest dangers with doing machine learning splits the wrong way -- <b> you will develop a false sense of confidence in how good your model is! </b>

Copyright 2018 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.